In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import print_function, division
import seaborn; seaborn.set()

In [4]:
from bot import sendMessage

In [3]:
r_train = pd.read_csv('data/r_train.csv', index_col=0)
r_test = pd.read_csv('data/r_test.csv', index_col=0)
r_target = pd.read_csv('data/target.csv', index_col=0)['0']

In [4]:
droplist = set(['ind_var6', 'saldo_medio_var17_hace3', 'delta_imp_trasp_var33_out_1y3', 
                'saldo_medio_var17_hace2', 'ind_var20_0', 'delta_imp_reemb_var33_1y3',
                'delta_imp_reemb_var13_1y3', 'num_var13_medio_0', 'ind_var20', 'ind_var18_0',
                'delta_imp_amort_var18_1y3', 'delta_imp_amort_var34_1y3', 'saldo_medio_var12_ult1',
                'delta_imp_aport_var13_1y3', 'delta_imp_venta_var44_1y3', 'saldo_medio_var17_ult3',
                'num_meses_var13_medio_ult3', 'delta_num_aport_var33_1y3', 'delta_num_aport_var17_1y3',
                'delta_num_compra_var44_1y3', 'imp_reemb_var33_ult1', 'delta_imp_trasp_var17_out_1y3',
                'imp_reemb_var17_hace3', 'ind_var34_0', 'delta_imp_trasp_var17_in_1y3'])

droplist = ['imp_op_var39_efect_ult3', 'num_op_var41_ult3', 'imp_op_var39_efect_ult1', 
            'delta_imp_aport_var13_1y3', 'delta_imp_reemb_var33_1y3', 'num_trasp_var33_out_ult1', 
            'ind_var18_0', 'delta_imp_reemb_var13_1y3', 'saldo_medio_var13_medio_ult3', 
            'ind_var20_0', 'imp_trasp_var17_in_hace3', 'num_op_var41_ult1', 'num_var1_0', 
            'num_var44', 'ind_var40_0', 'ind_var13_largo_0', 'num_var34_0', 'ind_var24', 
            'num_op_var41_efect_ult1', 'saldo_var24', 'imp_op_var41_ult1', 'ind_var20', 
            'ind_var39_0', 'num_var40', 'num_trasp_var17_in_ult1', 'num_var8_0', 
            'saldo_medio_var17_ult3', 'ind_var6_0', 'num_var8', 'num_var6', 'num_var13_medio_0', 
            'num_var4', 'ind_var14', 'num_var1', 'saldo_var1', 'saldo_medio_var33_ult3', 
            'num_reemb_var17_hace3', 'num_op_var41_efect_ult3', 'delta_num_venta_var44_1y3', 
            'delta_num_aport_var33_1y3', 'saldo_medio_var17_hace3', 'saldo_medio_var17_hace2', 
            'ind_var34_0', 'saldo_medio_var33_ult1', 'num_var45_ult3', 'saldo_medio_var12_ult1', 
            'num_var18_0', 'saldo_medio_var13_medio_hace2', 'num_reemb_var33_ult1', 'ind_var13_corto_0', 
            'ind_var13', 'num_op_var41_hace3', 'num_op_var41_hace2', 'ind_var7_emit_ult1', 
            'num_var12', 'saldo_medio_var13_largo_ult1', 'ind_var24_0', 'saldo_var17', 
            'num_var5', 'delta_num_compra_var44_1y3', 'saldo_var26', 'saldo_var34', 
            'saldo_medio_var13_corto_ult1', 'saldo_var13_corto', 'saldo_var18', 
            'delta_num_aport_var17_1y3', 'num_meses_var13_medio_ult3', 'num_trasp_var17_out_ult1', 
            'ind_var44_0', 'imp_trasp_var33_out_ult1', 'num_var40_0', 'ind_var1', 'num_var13_corto']
droplist = []

In [5]:
data = r_train.drop(droplist, axis=1)

In [6]:
%%time
dc = data.corr()

Wall time: 22.9 s


In [7]:
#Find correlations
def bad_feature_pairs(corr, columns, th):
    bad_fts = dict()
    bad_count = {ft:0 for ft in columns}
    for ft1 in columns:
        for ft2 in columns:
            if ft1 == ft2: 
                continue
            if corr[ft1][ft2] > th:
                pair = tuple(sorted([ft1, ft2]))
                if pair not in bad_fts:
                    bad_fts[pair] = corr[ft1][ft2]
                    bad_count[ft1] += 1
                    bad_count[ft2] += 1
    return bad_fts, bad_count

corr_th = .95
bad_fts, bad_count = bad_feature_pairs(dc, dc.columns, corr_th)
print(len(bad_fts))

148


In [8]:
for fts in sorted(bad_fts.keys(), key=lambda x: -bad_fts[x]):
    print(fts[0], fts[1], bad_fts[fts])

ind_var6 num_var6 1.00000000001
delta_imp_trasp_var33_out_1y3 num_trasp_var33_out_ult1 1.0
delta_imp_reemb_var33_1y3 num_reemb_var33_ult1 1.0
ind_var20_0 num_var20_0 1.0
delta_imp_reemb_var13_1y3 num_reemb_var13_ult1 1.0
ind_var20 num_var20 1.0
ind_var13_medio_0 num_var13_medio_0 1.0
num_meses_var13_medio_ult3 num_var13_medio_0 1.0
delta_imp_amort_var18_1y3 num_var18_0 1.0
ind_var18_0 num_var18_0 1.0
delta_imp_amort_var34_1y3 num_var34_0 1.0
delta_imp_aport_var13_1y3 delta_num_aport_var13_1y3 1.0
delta_imp_venta_var44_1y3 delta_num_venta_var44_1y3 1.0
delta_imp_aport_var33_1y3 delta_num_aport_var33_1y3 1.0
delta_imp_aport_var17_1y3 delta_num_aport_var17_1y3 1.0
ind_var13_medio_0 num_meses_var13_medio_ult3 1.0
delta_imp_compra_var44_1y3 delta_num_compra_var44_1y3 1.0
delta_imp_amort_var18_1y3 ind_var18_0 1.0
imp_reemb_var33_ult1 num_reemb_var33_ult1 1.0
delta_imp_trasp_var17_out_1y3 num_trasp_var17_out_ult1 1.0
imp_reemb_var17_hace3 num_reemb_var17_hace3 1.0
ind_var34_0 num_var34_0 1.0


In [9]:
for ft in sorted(bad_count, key=lambda x: -bad_count[x]):
    if bad_count[ft] > 0:
        print(ft, bad_count[ft])

saldo_var17 6
saldo_medio_var17_hace2 6
saldo_medio_var17_ult1 6
saldo_medio_var17_ult3 6
saldo_medio_var17_hace3 6
imp_aport_var17_hace3 6
saldo_var31 6
saldo_var24 4
saldo_medio_var12_ult1 4
saldo_var12 4
num_var13_corto 4
num_var13_corto_0 4
saldo_medio_var13_medio_hace2 4
ind_var13_corto_0 4
ind_var13_corto 4
saldo_medio_var12_ult3 4
saldo_var42 4
num_meses_var13_corto_ult3 4
num_var13_medio_0 3
num_var40 3
num_var1 3
num_var40_0 3
ind_var1 3
ind_var40_0 3
num_meses_var13_medio_ult3 3
ind_var13_medio_0 3
ind_var13_0 3
num_var1_0 3
ind_var40 3
ind_var1_0 3
num_var13 3
delta_imp_reemb_var33_1y3 2
ind_var13 2
ind_var12 2
saldo_var18 2
imp_trasp_var33_out_ult1 2
saldo_medio_var33_ult1 2
num_var6 2
num_reemb_var33_ult1 2
saldo_medio_var33_ult3 2
saldo_var13_corto 2
ind_var6 2
saldo_medio_var13_medio_ult3 2
num_trasp_var33_out_ult1 2
saldo_var1 2
saldo_var6 2
ind_var34_0 2
num_var13_0 2
saldo_medio_var13_corto_ult1 2
saldo_medio_var13_corto_ult3 2
ind_var18_0 2
imp_amort_var18_ult1 2
num

In [10]:
correlated = set(map(lambda x: x[0], bad_fts) + map(lambda x: x[1], bad_fts))
len(correlated)

173

In [11]:
to_drop = set()
drop_type = 'identical'
while True:
    bf, bc = bad_feature_pairs(dc, set(dc.columns).difference(to_drop), corr_th)
    #print(bc)
    if drop_type == 'identical':
        ident = filter(lambda x: bf[x] >= 1., bf.keys())
        if len(ident) == 0:
            break
        to_drop.add(ident[0][0])
        print(ident[0][0], bf[ident[0]])
    else:
        bcs = sorted(bc, key=lambda x: -bc[x])
        print(bcs[0], bc[bcs[0]])
        if bc[bcs[0]] == 0:
            break
        to_drop.add(bcs[0])
print(len(to_drop), to_drop)

delta_imp_venta_var44_1y3 1.0
delta_imp_reemb_var33_1y3 1.0
ind_var13_medio_0 1.0
delta_imp_aport_var13_1y3 1.0
ind_var20 1.0
delta_imp_aport_var33_1y3 1.0
delta_imp_amort_var34_1y3 1.0
ind_var6 1.00000000001
delta_imp_aport_var17_1y3 1.0
ind_var18_0 1.0
delta_imp_trasp_var33_out_1y3 1.0
delta_imp_amort_var18_1y3 1.0
ind_var20_0 1.0
num_meses_var13_medio_ult3 1.0
delta_imp_reemb_var13_1y3 1.0
15 set(['delta_imp_aport_var13_1y3', 'delta_imp_reemb_var33_1y3', 'delta_imp_venta_var44_1y3', 'ind_var20', 'ind_var18_0', 'delta_imp_amort_var34_1y3', 'delta_imp_reemb_var13_1y3', 'delta_imp_aport_var17_1y3', 'ind_var20_0', 'num_meses_var13_medio_ult3', 'delta_imp_trasp_var33_out_1y3', 'delta_imp_amort_var18_1y3', 'ind_var13_medio_0', 'ind_var6', 'delta_imp_aport_var33_1y3'])


In [14]:
c_train = r_train.drop(to_drop, axis=1)
c_test = r_test.drop(to_drop, axis=1)
c_test.to_csv('./data/c_test.csv')
c_train.to_csv('./data/c_train.csv')

In [5]:
target = pd.read_csv('./data/target.csv', index_col=0)['0']
c_test = pd.read_csv('./data/c_test.csv', index_col=0)
c_train = pd.read_csv('./data/c_train.csv', index_col=0)

In [7]:
for l in range(10000, len(target), 1000):
    print(target[:l].mean())

0.0388
0.0383636363636
0.0391666666667
0.0396923076923
0.0398571428571
0.0403333333333
0.0400625
0.0401764705882
0.0405555555556
0.0401052631579
0.04065
0.0400476190476
0.0402272727273
0.0407826086957
0.0412083333333
0.04108
0.0408076923077
0.0405925925926
0.0404285714286
0.0406206896552
0.0402666666667
0.0401612903226
0.04
0.0398484848485
0.0396764705882
0.0394285714286
0.0393888888889
0.0391891891892
0.0389736842105
0.0387948717949
0.0388
0.0387073170732
0.0386666666667
0.0387906976744
0.0386363636364
0.0386444444444
0.0386086956522
0.0387021276596
0.03875
0.0386326530612
0.03872
0.0386274509804
0.0387115384615
0.038679245283
0.0386666666667
0.0387090909091
0.0386607142857
0.0386140350877
0.0385
0.0385423728814
0.0386333333333
0.0386393442623
0.0386935483871
0.0387777777778
0.038828125
0.0389692307692
0.0390757575758
0.039223880597
0.0392058823529
0.0392753623188
0.0393285714286
0.0393661971831
0.0393472222222
0.0393424657534
0.0394864864865
0.03952
0.0395526315789


In [5]:
from sklearn.cross_validation import StratifiedKFold as SSS
sss = SSS(target, 5)
for train_idx, test_idx in sss:
    c_train80 = c_train.loc[train_idx]
    c_train20 = c_train.loc[test_idx]
    target80 = target.loc[train_idx]
    target20 = target.loc[test_idx]
    break
c_train80.to_csv('./data/c_train80.csv')
c_train20.to_csv('./data/c_train20.csv')
target80.to_csv('./data/target80.csv', header=True)
target20.to_csv('./data/target20.csv', header=True)
c_train80[:].isnull().sum().sum()

0L

In [23]:
print(target.mean(), target20.mean(), target80.mean())

0.0395685345962 0.0395922393949 0.0395626079092


In [86]:
c_train80 = pd.read_csv('./data/c_train80.csv', index_col=0)
c_train20 = pd.read_csv('./data/c_train20.csv', index_col=0)
target80 = pd.read_csv('./data/target80.csv', index_col=0)['0']
target20 = pd.read_csv('./data/target20.csv', index_col=0)['0']

In [10]:
def smooth_target_mean(train, test, target, C):
    res = np.zeros(test.shape)
    mean_target = target.mean()
    for val in train.unique():
        cat_size = train.value_counts()[val]
        cat_mean = target.iloc[np.where(train == val)].mean()
        res[np.where(test == val)] = float(cat_mean * cat_size + mean_target * C) / float(cat_size + C)
    return res

def smooth_target_mean_cv(train, target, C, cv=None):
    if cv == None:
        cv = [([x for x in range(len(train))], [x for x in range(len(train))])]
    if type(cv) == int:
        from sklearn.cross_validation import StratifiedKFold
        cv = StratifiedKFold(target, cv)
    #(среднее значение * размер категории + глобальное среднее значение * C) / (размер категории + С)
    res = np.zeros(train.shape)
    for trx, tsx in cv:
        Xtrain, Ytrain = train.iloc[trx], target.iloc[trx]
        Xtest, Ytest = train.iloc[tsx], target.iloc[tsx]
        
        mean_target = Ytrain.mean()
        cv_res = res[tsx]
        for val in Xtrain.unique():
            cat_size = Xtrain.value_counts()[val]
            cat_mean = Ytrain.iloc[np.where(Xtrain == val)].mean()
            #map(print, map(type, [cat_size]))
            #print(val)
            cv_res[np.where(Xtest == val)] = float(cat_mean * cat_size + mean_target * C) / float(cat_size + C)
        res[tsx] = cv_res
    return res

In [7]:
uniqs = c_train80.apply(lambda x: len(x.unique())).sort_values(ascending=False)
uniqs[:15]

var38                      46677
saldo_medio_var5_ult3      14522
saldo_var30                13942
saldo_var42                12929
saldo_medio_var5_ult1      12251
saldo_medio_var5_hace2     12074
saldo_var5                  9658
imp_op_var39_comer_ult3     7435
imp_op_var41_comer_ult3     7330
imp_op_var39_ult1           6669
saldo_medio_var5_hace3      6591
imp_op_var41_ult1           6581
imp_op_var39_comer_ult1     6195
imp_op_var41_comer_ult1     6097
saldo_var37                 3327
dtype: int64

In [9]:
good_cols = filter(lambda x: uniqs[x] < 5000, c_train80.columns)
print (len(good_cols), uniqs[good_cols].sum() / uniqs.sum())

278 0.217558630655


In [13]:
#%%time
import sys
for C in range(10, 60, 10):
    pass#print (C)

stm_train80 = c_train80.copy()
C = 10
for i in range(len(good_cols)):
    col = good_cols[i]
    #print(col, c_train80[col].shape, target80.shape)
    stm_train80[col + '_stm%d' % C] = smooth_target_mean_cv(c_train80[col], target80, C=C, cv=10)
    print('\r%.2f%%' % (100 * uniqs[good_cols[:i + 1]].sum() / uniqs[good_cols].sum()), col, end=' ')
    sys.stdout.flush()

n0 1.000000% 

In [15]:
%%time
stm_train20 = c_train20.copy()
C = 10
for i in range(len(good_cols)):
    col = good_cols[i]
    stm_train20[col + '_stm%d' % C] = smooth_target_mean(c_train80[col], c_train20[col], target80, C=C)
    print('\r%.2f%%' % (100 * uniqs[good_cols[:i + 1]].sum() / uniqs[good_cols].sum()), col, end=' ')
    sys.stdout.flush()

100.000000%n0 Wall time: 2min 5s


In [ ]:
sendMessage('olou')

In [16]:
stm_train20.to_csv('./data/stm_train20.csv')
stm_train80.to_csv('./data/stm_train80.csv')

In [17]:
print(stm_train80.shape, stm_train20.shape, target80.shape, target20.shape)

(60815, 570) (15205, 570) (60815L,) (15205L,)


In [19]:
%%time
params = [(col, c_train[col], c_test[col], target, 20) for col in c_test.columns[:4]]

Wall time: 1e+03 µs


In [ ]:
import multiprocessing as mp
pool = mp.Pool(4)
def zip_smt(args):
    return args[0], smooth_mean_target(*args[1:])

try:
    res = pool.map(zip_smt, params)
    for col, feat in res:
        c_test[col + '_stm'] = feat
finally:
    pool.terminate()
    pool.stop()
    del pool

In [ ]:
def devar(col):
    spl = col.split('_')
    for i in range(len(spl)):
        if spl[i].startswith('var'):
            spl[i] = 'varN'
    return '_'.join(spl)

from collections import Counter
Counter(map(devar, c_train.columns));